<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  completa tu nombre aquí :D

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**

- Es de filtrado colaborativo ya que mediante los ratings de otros usuarios a un item especifico, es capas de predecir el rating que dara un usuario que aun no a evaluado aquel item especifico.

-  Esta basado en items ya que compara directamente similitudes entre items para predecir uno nuevo.

- Recibe todos los ratings en comun con otros usuarios que hayan revisado la pelicula/serie a predecir, por item al que yo le haga echo un review, luego calcula por cada item que haya revisado el diferencial con la pelicula a predecir y lo pondera por los usuarios utilizados, se suman todos los diferenciales y se divide por la cantidad de usuarios utilizados.

- Dependiendo de la cantidad de ratings iniciales puede ser un cambio marginal pero en teoria el numero a predecir cambia si el item al cual se le agrego el rating yo lo tengo evaluado ya que ahora seria necesario considerarlo para la prediccion.

- Mientras menos ratings tenga el usuario sera menos la cantidad de items en comun que existan con otros usuarios por lo que la prediccion no sera representativa al no tener suficientes comparaciones de items.




# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    338      0 --:--:--  0:00:01 --:--:--   338
100 1546k  100 1546k    0     0   942k      0  0:00:01  0:00:01 --:--:--  942k


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    466      0 --:--:-- --:--:-- --:--:--   466
100  385k  100  385k    0     0   300k      0  0:00:01  0:00:01 --:--:--  188M


In [3]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    609      0 --:--:-- --:--:-- --:--:--   609
100  230k  100  230k    0     0   230k      0  0:00:01  0:00:01 --:--:--  225M


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [4]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 234 kB 4.0 MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [5]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [6]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [7]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [8]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:** Calcula los diferenciales entre los usuarios para luego poder hacer las predicciones

## Paso 6:

Llego la hora de predecir el rating.

In [9]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [10]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [26]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [17]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 1168]

,userid,itemid,rating,timestamp
6664,85,1168,3,882995908
9646,122,1168,4,879270902
20141,269,1168,2,891448386
21600,280,1168,5,891702544
29685,363,1168,2,891496587
30647,378,1168,3,880333383
30899,380,1168,3,885479833
31698,389,1168,3,880088717
32183,393,1168,3,889729346
34386,416,1168,4,886318953


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** Se explican ya que al momento de predecir se utilizan todas las peliculas que el usuario haya evaluado en conjunto a otros usuarios, por lo que si existen muchas peliculas o hay diferencias muy grandes con los otros usuarios si bien los valores predichos seran cercanos no seran necesariamente iguales. Ademas existen factores externos que pueden afectar el rating que el usuario dio a la pelicula.

In [18]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [19]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [20]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** Al recomendar 7 peliculas al usuario de id 85 las recomendaciones son completamente diferentes 

In [28]:
# Escribe el nuevo codigo aqui
reclist_slopeone = mySlopeOne.recommend("85", 7)
print('Lista de items según ID:', reclist_slopeone)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']


Lista de items según ID: ['1536', '1500', '44', '43', '41', '39', '814']
Lista de items por nombre:


movieid
1536             Aiqing wansui (1994)
1500        Santa with Muscles (1996)
44           Dolores Claiborne (1994)
43                  Disclosure (1994)
41               Billy Madison (1995)
39                Strange Days (1995)
814     Great Day in Harlem, A (1994)
Name: title, dtype: object

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:** se puede ver claramente que son completamente diferentes, la primera tabla (Most popular) claramente tiene peliculas mas mainstream que la segunda por lo que el usuario 44 tiene gustos diferentes a lo popular, en cambio la tabla de slope one representa de forma mas real los gustos del usuario 44, ya que de las peliculas que no a visto las que mas le pueden interesar segun slope no son las mas populares entre los usuarios. otra diferencia importante son los generos de las peliculas ya que las mas populares suelen ser de accion o scifi, mientras que las recomendadas por slope suelen ser dramas o romance.

In [37]:
# Escribe el nuevo codigo aqui
print('Most Popular')
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)  
most_popular.train()
ranking = [int(r) for r in most_popular.recommend(str(44), 15, includeRated=False)]
rec = np.array(ranking).astype(int)
print('Recommendation for user {}: {}'.format(44, ranking))
print(info_file.loc[rec]['title'])
print()
print('Slope one')
reclist_slopeone = mySlopeOne.recommend("44", 15)
print('Lista de items según ID:', reclist_slopeone)
recmovies_slopeone = np.array(reclist_slopeone).astype(int)
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']


Most Popular
Recommendation for user 44: [50, 100, 181, 286, 1, 121, 300, 127, 7, 98, 172, 56, 237, 117, 222]
movieid
50                     Star Wars (1977)
100                        Fargo (1996)
181           Return of the Jedi (1983)
286         English Patient, The (1996)
1                      Toy Story (1995)
121       Independence Day (ID4) (1996)
300                Air Force One (1997)
127               Godfather, The (1972)
7                 Twelve Monkeys (1995)
98     Silence of the Lambs, The (1991)
172     Empire Strikes Back, The (1980)
56                  Pulp Fiction (1994)
237                Jerry Maguire (1996)
117                    Rock, The (1996)
222     Star Trek: First Contact (1996)
Name: title, dtype: object

Slope one
Lista de items según ID: ['1656', '1064', '1643', '1642', '1625', '1599', '1512', '1536', '1500', '1467', '1463', '1449', '1367', '1293', '868']
Lista de items por nombre:


movieid
1656                        Little City (1998)
1064                          Crossfire (1947)
1643                         Angel Baby (1995)
1642                  Some Mother's Son (1996)
1625                         Nightwatch (1997)
1599             Someone Else's America (1995)
1512    World of Apu, The (Apur Sansar) (1959)
1536                      Aiqing wansui (1994)
1500                 Santa with Muscles (1996)
1467      Saint of Fort Washington, The (1993)
1463                          Boys, Les (1997)
1449                    Pather Panchali (1955)
1367                              Faust (1994)
1293                           Star Kid (1997)
868                    Hearts and Minds (1996)
Name: title, dtype: object